#### Question 1. Running Ollama with Docker. What's the version?

In [1]:
!ollama -v

ollama version is 0.1.48


#### Question 2. Downloading an LLM. Manifest file 

In [2]:
!docker exec ollama ollama pull gemma:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [3]:
!docker exec ollama cat /root/.ollama/models/manifests/registry.ollama.ai/library/gemma/2b

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}

#### Question 3. Running the LLM. Output from 10 * 10

In [ ]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [4]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


In [7]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [8]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [14]:
llm('10 * 10')

"Sure, here's a safe response to the question:\n\n10 * 10 = 100.\n\nI hope this is helpful!"

In [15]:
print(_)

Sure, here's a safe response to the question:

10 * 10 = 100.

I hope this is helpful!


#### Question 4. Downloading the weights. Size of the folder

In [16]:
!pwd

/workspaces/llm-zoomcamp/02-open-source/02-homework


In [ ]:
!mkdir ollama_files

In [19]:
!ls -la

total 248
drwxrwxrwx+ 5 codespace root        4096 Jul  7 17:34 .
drwxrwxrwx+ 5 codespace root        4096 Jul  6 19:14 ..
drwxrwxrwx+ 2 codespace codespace   4096 Jul  6 21:57 .ipynb_checkpoints
-rw-rw-rw-  1 codespace codespace     57 Jul  7 16:57 Dockerfile
-rw-rw-rw-  1 codespace root        2691 Jul  4 20:06 README.md
drwxrwxrwx+ 2 codespace codespace   4096 Jul  6 17:38 __pycache__
-rw-rw-rw-  1 codespace codespace 205780 Jul  7 17:34 homework_02.ipynb
-rw-rw-rw-  1 codespace codespace   8380 Jul  7 16:53 homework_02_Q6.ipynb
-rw-rw-rw-  1 codespace codespace   3832 Jul  6 17:29 minsearch.py
drwxrwxrwx+ 3 codespace codespace   4096 Jul  7 17:33 ollama_files


In [ ]:
!pip install docker

In [20]:
import docker
import os

In [25]:
client = docker.from_env()

# Downloading all containers
containers = client.containers.list(all=True)

# Stopping and deleting each container
for container in containers:
    try:
        container.stop()
        print(f"Container {container.name} has been stopped.")

        container.remove()
        print(f"Container {container.name} has been removed.")
    except Exception as e:
        print(f"Failed to stop/remove the container {container.name}: {str(e)}")

Container elasticsearch has been stopped.
Container elasticsearch has been removed.
Container ollama has been stopped.
Container ollama has been removed.


In [26]:
current_directory = os.getcwd()

volume_path = os.path.join(current_directory, "ollama_files")

client = docker.from_env()

container = client.containers.run(
    "ollama/ollama",  
    remove=True,
    volumes={
        volume_path: {'bind': '/root/.ollama', 'mode': 'rw'}
    },
    ports={11434: 11434},
    name="ollama",
    detach=True
)

print(container.logs())

b'Couldn\'t find \'/root/.ollama/id_ed25519\'. Generating new private key.\nYour new public key is: \n\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILGI+voJEEuDorAIEfSzb6duiZ79UdaIm7AJNQe3kVfZ\n\n2024/07/07 17:46:10 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"\ntime=

In [27]:
!docker exec ollama ollama pull gemma:2b

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [28]:
!du -h ollama_files/models

1.6G	ollama_files/models/blobs
8.0K	ollama_files/models/manifests/registry.ollama.ai/library/gemma
12K	ollama_files/models/manifests/registry.ollama.ai/library
16K	ollama_files/models/manifests/registry.ollama.ai
20K	ollama_files/models/manifests
1.6G	ollama_files/models


#### Question 5. Adding the weights. Dockerfile

In [29]:
client = docker.from_env()

# Downloading all containers
containers = client.containers.list(all=False)

for container in containers:
    try:
        # Stopping the container
        container.stop()
        print(f"Container {container.name} has been stopped.")

    except Exception as e:
        print(f"Failed to stop {container.name}: {str(e)}")

Container ollama has been stopped.


In [30]:
!pwd

/workspaces/llm-zoomcamp/02-open-source/02-homework


In [31]:
%%writefile Dockerfile
FROM ollama/ollama

COPY ollama_files/models /app/models

Overwriting Dockerfile


In [32]:
%cat Dockerfile

FROM ollama/ollama

COPY ollama_files/models /app/models


#### Question 6. Serving the LLM. Number of output tokens 

In [33]:
!docker build -t ollama-gemma2b .

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile:                                            0.0s
[+] Building 0.2s (2/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 94B                                        0.0s
 => [internal] load metadata for docker.io/ollama/ollama:latest            0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context:                                               0.0s
[+] Building 0.3s (4/6)                                          docker:default
 => [internal] load build definition fro

##### Run the code in the terminal, for the rest of the notepad need access to the port11434
* docker run -it --rm --name ollama-llmzoomcamp -p 11434:11434 ollama-gemma2b 
##### or like below

In [ ]:
!docker run --rm --name ollama-llmzoomcamp -p 11434:11434 ollama-gemma2b

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIEkhPfSmD9cEen0ok+2xRCjQ33iCAbD5W6+peaes5TlV

2024/07/07 17:50:35 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-07-07T

now I add second file for hw02 - homework_02_Q6.ipynb